In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split # 데이터 분리를 위해 추가
from sklearn.metrics import accuracy_score         # 정확도 계산을 위해 추가
import joblib
import sys

print("--- 1. 라이브러리 임포트 ---")

# 1. 데이터 로드
# 파일 경로를 이전에 생성한 파일로 수정했습니다.
file_path = 'final_2024_processed_data.csv'

try:
    df = pd.read_csv(file_path)
    print(f"--- 2. '{file_path}' 데이터 로드 완료 ---")
    print(f"원본 데이터 크기: {df.shape}")

    # 2. 데이터 전처리: NaN 값 처리 및 타겟 변수 정수형 변환
    # 모델 학습을 위해 NaN 값을 가진 행을 제거합니다.
    df.dropna(inplace=True)
    print(f"--- 3. NaN 값 제거 후 데이터 크기: {df.shape} ---")

    # 'tsunami' 컬럼을 분류에 적합한 정수형(int)으로 변환합니다.
    df['tsunami'] = df['tsunami'].astype(int)
    print("--- 4. 'tsunami' 컬럼을 정수형(int)으로 변환 ---")

    # 3. 특징(X)과 타겟(y) 변수 분리
    X = df.drop('tsunami', axis=1)
    y = df['tsunami']
    print("--- 5. 특징(X)과 타겟(y) 분리 완료 ---")

    # 데이터가 비어 있는지 확인
    if X.empty or y.empty:
        print("오류: NaN 값을 제거한 후 데이터가 남아있지 않습니다.")
        sys.exit()

    # 타겟 클래스가 2개 이상인지 확인 (분류를 위해)
    if y.nunique() < 2:
        print("경고: 타겟 클래스가 1개뿐입니다. stratify 없이 분할합니다.")
        stratify_param = None
    else:
        stratify_param = y


    # 4. 학습 데이터와 테스트 데이터 분리 (★ 추가된 부분)
    # 모델을 평가하기 위해 데이터를 학습용(80%)과 테스트용(20%)으로 분리합니다.
    # stratify=y : 타겟(y)의 클래스 비율을 유지하면서 데이터를 나눕니다.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,    # 20%를 테스트용으로 사용
        random_state=42,  # 결과를 동일하게 재현하기 위함
        stratify=stratify_param # 'tsunami' 비율을 맞춤
    )
    print(f"--- 6. 데이터 분리 완료 (학습용: {X_train.shape[0]}개, 테스트용: {X_test.shape[0]}개) ---")

    # 5. RandomForest 분류기 모델 초기화
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    print("--- 7. RandomForest 모델 초기화 완료 ---")

    # 6. 모델 학습 (★ 수정된 부분: 학습용 데이터로 학습)
    # X(특징)와 y(타겟) 데이터를 사용하여 모델을 학습시킵니다.
    model.fit(X_train, y_train)
    print("--- 8. 모델 학습 완료 ---")

    # 7. 모델 평가 및 정확도 출력 (★ 추가된 부분)
    # 테스트 데이터로 예측을 수행합니다.
    y_pred = model.predict(X_test)

    # 정확도를 계산합니다.
    accuracy = accuracy_score(y_test, y_pred)

    print("\n--- 9. 모델 평가 (정확도) ---")
    print(f"테스트 데이터 정확도: {accuracy * 100:.2f}%") # 정확도를 퍼센트로 출력

    # 8. 학습된 모델을 파일로 저장
    model_filename = 'random_forest_tsunami_model.joblib'
    joblib.dump(model, model_filename)

    print(f"\n[최종 완료] 모델이 '{model_filename}' 파일로 성공적으로 저장되었습니다.")

except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인하세요.")
except KeyError:
    print("오류: 'tsunami' 컬럼을 찾을 수 없습니다. 원본 파일의 컬럼을 확인하세요.")
except ValueError as e:
    print(f"데이터 처리 중 오류가 발생했습니다: {e}")
except Exception as e:
    print(f"예상치 못한 오류가 발생했습니다: {e}")

--- 1. 라이브러리 임포트 ---
--- 2. 'final_2024_processed_data.csv' 데이터 로드 완료 ---
원본 데이터 크기: (5872, 6)
--- 3. NaN 값 제거 후 데이터 크기: (5872, 6) ---
--- 4. 'tsunami' 컬럼을 정수형(int)으로 변환 ---
--- 5. 특징(X)과 타겟(y) 분리 완료 ---
--- 6. 데이터 분리 완료 (학습용: 4697개, 테스트용: 1175개) ---
--- 7. RandomForest 모델 초기화 완료 ---
--- 8. 모델 학습 완료 ---

--- 9. 모델 평가 (정확도) ---
테스트 데이터 정확도: 98.72%

[최종 완료] 모델이 'random_forest_tsunami_model.joblib' 파일로 성공적으로 저장되었습니다.
